In [ ]:
import random
import numpy as np
import pandas as pd

from efin import *
from functools import partial
from esn_tarnet import *
from feature_select import *
from s_learner import *
from t_learner import *
from tarnet import *
from evaluate import *

def set_seed(seed):
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
set_seed(42)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

In [ ]:
#读取特征列表
import pickle
with open('./feature_list_bonus_pdp_id_selected.pkl', 'rb') as f:
    feature_list = pickle.load(f)
print(len(feature_list))

In [ ]:
#读取特征列表
import pickle
with open('./feature_list_bonus_pdp_id_selected_discrete.pkl', 'rb') as f:
    feature_list_discrete = pickle.load(f)
print(len(feature_list_discrete))

In [ ]:
#读取特征列表
import pickle
with open('./feature_list_bonus_pdp_id_selected_discrete_size.pkl', 'rb') as f:
    discrete_size_cols = pickle.load(f)
print(discrete_size_cols)

In [ ]:
model_list = {}

In [ ]:

model = ESN_Tarnet(
    input_dim=len(feature_list), discrete_size_cols=discrete_size_cols,embedding_dim=3,share_dim=64,
                 share_hidden_dims =[256,128,64,64],
                 base_hidden_dims=[64,32,32,16],output_activation_base=torch.nn.Sigmoid(),
                 ipw_hidden_dims=[256,128,64,64],output_activation_ipw=None,
                 share_hidden_func = torch.nn.ELU(),base_hidden_func = torch.nn.ELU(), ipw_hidden_func = torch.nn.ELU(),
                 task = 'classification',classi_nums=2, treatment_label_list=[0,1,2,3],model_type='ESN_Tarnet',device=device
)
checkpoint = torch.load('./bonus_pdp_id_esn_tarnet_checkpoint_256_69.pth', map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model_list['ESN_Tarnet_256'] = model

model = Tarnet(
    input_dim=len(feature_list), discrete_size_cols=discrete_size_cols,embedding_dim=3,share_dim=64,
                 share_hidden_dims =[256,128,64,64],
                 base_hidden_dims=[64,32,32,16],output_activation_base=torch.nn.Sigmoid(),
                 share_hidden_func = torch.nn.ELU(),base_hidden_func = torch.nn.ELU(), 
                 task = 'classification',classi_nums=2, treatment_label_list=[0,1,2,3],model_type='ESN_Tarnet',device=device
)
checkpoint = torch.load('./bonus_pdp_id_checkpoint_tarnet_256_75.pth', map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model_list['Tarnet_256'] = model

model = Tarnet(
    input_dim=len(feature_list), discrete_size_cols=discrete_size_cols,embedding_dim=3,share_dim=64,
                 share_hidden_dims =[256,128,64,64],
                 base_hidden_dims=[64,32,32,16],output_activation_base=torch.nn.Sigmoid(),
                 share_hidden_func = torch.nn.ELU(),base_hidden_func = torch.nn.ELU(), 
                 task = 'classification',classi_nums=2, treatment_label_list=[0,1,2,3],model_type='ESN_Tarnet',device=device
)
checkpoint = torch.load('./bonus_pdp_id_checkpoint_tarnet_512_89.pth', map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model_list['Tarnet_512'] = model


model = Slearner(
    input_dim=len(feature_list), discrete_size_cols=discrete_size_cols,embedding_dim=3,
                 base_hidden_dims=[64,32,32,16],output_activation_base=None,base_hidden_func = torch.nn.ELU(), 
                 task = 'classification',classi_nums=2, treatment_label_list=[0,1,2,3],model_type='Tlearner',device=device
)
checkpoint = torch.load('./bonus_pdp_id_slearn_checkpoint_512_94.pth', map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model_list['Slearner'] = model

model = Tlearner(
    input_dim=len(feature_list), discrete_size_cols=discrete_size_cols,embedding_dim=3,
                 base_hidden_dims=[64,32,32,16],output_activation_base=None,base_hidden_func = torch.nn.ELU(), 
                 task = 'classification',classi_nums=2, treatment_label_list=[0,1,2,3],model_type='Tlearner',device=device
)
checkpoint = torch.load('./bonus_pdp_id_tlearn_checkpoint_256_16.pth', map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model_list['Tlearner'] = model

In [ ]:
df = pd.read_parquet('/mlx_devbox/users/wangyuxin.huoshan/playground/bonus_train_data/bonus_pdp_id_test_id',columns=feature_list+['enter_ab_date','is_treatment','gmv','roas_label']).fillna(0)

In [ ]:
X_train = torch.tensor(df[feature_list].values, dtype=torch.float32).to(device)
X_discrete = torch.tensor(df[feature_list_discrete].values, dtype=torch.float32).to(device)
X_continuous = torch.tensor(df[[_ for _ in feature_list if _ not in feature_list_discrete]].values, dtype=torch.float32).to(device)

In [ ]:
uplift_prediction = {}
for each in model_list.keys():
    model = model_list[each]
    model.to(device)
    model.eval()
    with torch.no_grad(): 
        uplift_predictions,y_preds,*eps = model(X_train, None, X_discrete=X_discrete, X_continuous=X_continuous)
    uplift_prediction[each] = torch.cat(uplift_predictions,dim=1).detach().cpu().numpy()


In [ ]:
# for treatment in [0,1,2,3]:
#     print(treatment)
#     df['y_predictions'] = y_prediction[:,treatment]
#     df_tmp = df[df['is_treatment'].isin([treatment])]
#     cacu_auc(df=df_tmp,y_true='roas_label',y_predictions='y_predictions',is_discreate=True)


In [ ]:
for treatment in [1,2,3]:
    print(treatment)
    for each in uplift_prediction.keys():
        print(each)
        df['uplift_predictions_'+each] = uplift_prediction[each][:,treatment-1]
 
    df_tmp = df[df['is_treatment'].isin([0,treatment])]
    df_tmp['is_treatment'] = df_tmp['is_treatment'].apply(lambda x: 1 if x == treatment else 0)
    df_tmp['random'] = np.random.rand(df_tmp.shape[0])

    evaluate(df=df_tmp[['roas_label','is_treatment','random'] + ['uplift_predictions_'+_ for _ in uplift_prediction.keys()]],y_true='roas_label',treatment='is_treatment',divide_feature=None)


    